In [4]:
import chromedriver_autoinstaller
chromedriver_autoinstaller.install()

'c:\\Users\\manya\\miniconda3\\envs\\proyecto_final\\lib\\site-packages\\chromedriver_autoinstaller\\114\\chromedriver.exe'

In [5]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from dotenv import load_dotenv
import time

import os
load_dotenv()


False

In [6]:
from bs4 import BeautifulSoup
import re as re
import time
import pandas as pd
import os
import numpy as np
import requests

import pymysql
import sqlalchemy as alch
from getpass import getpass

Pasos scrapping: 
1. Scrapear el link http://www.gardenate.com/plants/ para conseguir todos los nombres de plantas y guardarlos en una lista
2. Scrapear cualquier link de una planta para conseguir todas las zona climáticas y guardar el numero de la zona como key y el nombre de la zona en value en un diccionario.
3. A partir de la lista de plantas y el diccionario de zonas climáticas, crear una lista donde almacenamos los links de las páginas que queremos scrapear ya que siguen un mismo formato. 
4. Con la lista de urls de solo plantas (sin tener en cuenta las zonas climáticas), scrapeamos la parte de características de la planta y creamos un dataframe que contiene como columnas: nombre planta, sowing, spacing, compatibility y url de la imagen de la planta. 

In [7]:
def plant_names (url): 
    """Esta función sirve para obtener el nombre de todas las plantas usando la librería beautiful soup. La función toma como argumento un url
    donde aparecen todos los nombres de las plantas y devuelve una lista de nombre."""
    html = requests.get(url)
    soup = BeautifulSoup(html.content, "html.parser")

    plants_list = soup.find_all('div',{'class':'plant col-sm-12'})
    names_plants = [i.getText().split('(')[0].strip().replace (' ', '%2B') for i in plants_list] #Cambio formato replace para adecuarlo al formato que tiene el url

    return names_plants

names_plants = plant_names('http://www.gardenate.com/plants/')

In [8]:
print(names_plants)

['Amaranth', 'Angelica', 'Artichokes', 'Asparagus', 'Asparagus%2BPea', 'Basil', 'Beans%2B-%2Bclimbing', 'Beans%2B-%2Bdwarf', 'Beetroot', 'Borage', 'Broad%2BBeans', 'Broccoli', 'Brussels%2Bsprouts', 'Burdock', 'Cabbage', 'Cape%2BGooseberry', 'Capsicum', 'Cardoon', 'Carrot', 'Cauliflower', 'Celeriac', 'Celery', 'Chicory', 'Chilli%2Bpeppers', 'Chinese%2Bcabbage', 'Chives', 'Choko/Chayote', 'Collards', 'Coriander', 'Corn%2BSalad', 'Cowpeas', 'Cucumber', 'Daikon', 'Dill', 'Eggplant', 'Endive', 'Fennel', 'Florence%2BFennel', 'French%2Btarragon', 'Garlic', 'Ginger', 'Horseradish', 'Jerusalem%2BArtichokes', 'Kale', 'Kohlrabi', 'Leeks', 'Lemon%2BBalm', 'Lettuce', 'Luffa', 'Marrow', 'Mint', 'Mizuna', 'Mustard%2Bgreens', 'NZ%2BSpinach', 'Okra', 'Onion', 'Oregano', 'Pak%2BChoy', 'Parsley', 'Parsnip', 'Peas', 'Potato', 'Pumpkin', 'Radish', 'Rhubarb', 'Rocket', 'Rockmelon', 'Rosella', 'Rosemary', 'Rutabaga', 'Sage', 'Salsify', 'Savory%2B-%2Bsummer%2Bsavory', 'Savory%2B-%2Bwinter%2Bsavory', 'Shallots

In [9]:
def zones (url):
    """Esta función sirve para obtener todas las zonas climáticas registradas en la página web y guardarlos en un diccionario donde la key es el value (numero
     de zona climática) y el value es el nombre de la zona climática. La función toma como argumento un link y devuelve un diccionario. """

    html = requests.get(url)
    soup = BeautifulSoup(html.content, "html.parser")

    filter = soup.find("select", class_="form-control zone-selector")
    
    # Diccionario vació para guardar las zonas climáticas
    climate_zones = {}

    # Extraer la opción elementos y completar el diccionario
    for option in filter.find_all("option"):
        zone_number = option["value"]
        zone_name = option.text.strip()
        climate_zones[zone_number] = zone_name

    return climate_zones

 
    

In [10]:
dict_climate = zones('https://www.gardenate.com/plant/Basil')

In [11]:
print(dict_climate)
print(names_plants)

{'17': 'Australia - arid', '1': 'Australia - cool/mountain', '3': 'Australia - sub-tropical', '2': 'Australia - temperate', '4': 'Australia - tropical', '61': 'Canada - Zone 2a Sub-Arctic', '62': 'Canada - Zone 2b Sub-Arctic', '60': 'Canada - Zone 3a Temperate Short Summer', '59': 'Canada - Zone 3b Temperate Warm Summer', '58': 'Canada - zone 4a Temperate Warm Summer', '57': 'Canada - Zone 4b Temperate Warm Summer', '56': 'Canada - Zone 5a Temperate Warm Summer', '55': 'Canada - Zone 5b Temperate Warm Summer', '54': 'Canada - Zone 6a Temperate Warm Summer', '53': 'Canada - Zone 6b Temperate Warm Summer', '52': 'Canada - Zone 7a Mild Temperate', '51': 'Canada - Zone 7b Mild Temperate', '50': 'Canada - Zone 8a Mild Temperate', '8': 'United Kingdom - cool/temperate', '9': 'United Kingdom - warm/temperate', '6': 'New Zealand - cool/mountain', '7': 'New Zealand - sub-tropical', '5': 'New Zealand - temperate', '107': 'USA - Zone 2a', '108': 'USA - Zone 2b', '109': 'USA - Zone 3a', '110': 'US

In [12]:
def get_urls_plants (list_):
    """Esta función sirve para crear los link de todas las plantas a partir de la lista de nombre de plantas. Toma como argumento una lista y devuelve
    otra lista de links"""
    list_url = []
    for i in list_:
        list_url.append(f'https://www.gardenate.com/plant/{i}')
    return list_url

In [13]:
url_plants = get_urls_plants (names_plants)

In [14]:
def get_urls_complete (list_, dict_): 
    """Esta función sirve para crear los links de todas las plantas según zona climática. Toma como argumento la lista de nombre de plantas y el diccionario
    de zonas climáticas y devuelve una lista de links"""
    list_url = []
    for i in list_:
        for j in dict_.keys():
            list_url.append(f'https://www.gardenate.com/plant/{i}?zone={j}')
    return list_url


In [16]:
list_url = get_urls_complete(names_plants, dict_climate)

In [17]:
list_url

['https://www.gardenate.com/plant/Amaranth?zone=17',
 'https://www.gardenate.com/plant/Amaranth?zone=1',
 'https://www.gardenate.com/plant/Amaranth?zone=3',
 'https://www.gardenate.com/plant/Amaranth?zone=2',
 'https://www.gardenate.com/plant/Amaranth?zone=4',
 'https://www.gardenate.com/plant/Amaranth?zone=61',
 'https://www.gardenate.com/plant/Amaranth?zone=62',
 'https://www.gardenate.com/plant/Amaranth?zone=60',
 'https://www.gardenate.com/plant/Amaranth?zone=59',
 'https://www.gardenate.com/plant/Amaranth?zone=58',
 'https://www.gardenate.com/plant/Amaranth?zone=57',
 'https://www.gardenate.com/plant/Amaranth?zone=56',
 'https://www.gardenate.com/plant/Amaranth?zone=55',
 'https://www.gardenate.com/plant/Amaranth?zone=54',
 'https://www.gardenate.com/plant/Amaranth?zone=53',
 'https://www.gardenate.com/plant/Amaranth?zone=52',
 'https://www.gardenate.com/plant/Amaranth?zone=51',
 'https://www.gardenate.com/plant/Amaranth?zone=50',
 'https://www.gardenate.com/plant/Amaranth?zone=8'

In [20]:
def characterisitics(list_):

    plants = []

    for i in list_: #url_plants
        html = requests.get(i)
        soup = BeautifulSoup(html.content, "html.parser")
        
        
        # Find the position of the last / and ?
        last_slash_index = i.rfind("/")
        question_mark_index = i.find("?")

        # Extract the desired part of the url
        name = i[last_slash_index + 1 : question_mark_index]
  
         
        # Extract sowing if available
        sowing_element = soup.find("li", {'class': 'sowing'})
        sowing = sowing_element.getText() if sowing_element else np.nan
        
        # Extract spacing if available
        spacing_element = soup.find("li", {'class': 'spacing'})
        spacing = spacing_element.getText() if spacing_element else np.nan
        
        # Extract harvest if available
        harvest_element = soup.find("li", {'class': 'harvest'})
        harvest = harvest_element.getText() if harvest_element else np.nan

        companion_element = soup.find("li", {'class': 'companion'})
        companion = companion_element.getText() if companion_element else np.nan

        avoid_element = soup.find("li", {'class': 'avoid'})
        avoid = avoid_element.getText() if avoid_element else np.nan



        image_link_element = soup.find("div", {"class":"image"})
        image_link = ('www.gardenate.com' + image_link_element.find("a").get('href') if image_link_element is not None else np.nan)

        dict_ = {'name':name ,'sowing':sowing, 'spacing':spacing, 'harvest':harvest, 'compatibility':companion,'avoid':avoid, 'image_link': image_link}

        plants.append(dict_)

        df = pd.DataFrame(plants)
        

    return df 


        

In [21]:
df_characteristics = characterisitics(url_plants)

In [22]:
df_characteristics.isna().sum()

name              0
sowing            4
spacing           4
harvest           4
compatibility     6
avoid            48
image_link        4
dtype: int64

In [43]:
url = 'https://www.gardenate.com/plant/Broccoli'
html = requests.get(url)
soup = BeautifulSoup(html.content, "html.parser")

table = soup.find('table')
headers = [header.text for header in table.find_all('th')]



In [44]:
print(table)
print (headers)

<table>
<tr>
<th>Jan</th>
<th>F<span class="hidden-xs">eb</span></th>
<th>M<span class="hidden-xs">ar</span></th>
<th>A<span class="hidden-xs">pr</span></th>
<th>M<span class="hidden-xs">ay</span></th>
<th>J<span class="hidden-xs">un</span></th>
<th>J<span class="hidden-xs">ul</span></th>
<th>A<span class="hidden-xs">ug</span></th>
<th>S<span class="hidden-xs">ep</span></th>
<th>O<span class="hidden-xs">ct</span></th>
<th>N<span class="hidden-xs">ov</span></th>
<th class="end">Dec</th>
</tr> <tr> <td class=""> </td> <td class=""> </td> <td class=""> </td> <td class="seed">S</td> <td class=""> </td> <td class=""> </td> <td class=""> </td> <td class=""> </td> <td class=""> </td> <td class=""> </td> <td class=""> </td> <td class="end"> </td> </tr> <tr> <td class=""> </td> <td class=""> </td> <td class=""> </td> <td class=""> </td> <td class=""> </td> <td class="transplant">T</td> <td class="transplant">T</td> <td class=""> </td> <td class=""> </td> <td class=""> </td> <td class=""> </td> 

In [45]:
data = []
for row in table.find_all('tr'):
    cells = [cell.text.strip() for cell in row.find_all('td')]
    if cells:
        data.append(cells)

In [46]:
print(data)

[['', '', '', 'S', '', '', '', '', '', '', '', ''], ['', '', '', '', '', 'T', 'T', '', '', '', '', '']]


In [47]:
url = "https://www.gardenate.com/plant/Broccoli?zone=2"
response = requests.get(url)
html_content = response.text

soup = BeautifulSoup(html_content, 'html.parser')

table = soup.find('table')
rows = table.find_all('tr')

data = []
headers = [header.text.strip() for header in rows[0].find_all('th')]

for row in rows[1:]:
    cells = [cell.text.strip() for cell in row.find_all('td')]
    data.append(cells)

df = pd.DataFrame(data, columns=headers)

In [49]:
data

[['S', 'S', 'S', '', '', '', '', 'S', 'S', 'S', 'S', 'S'],
 ['T', 'T', 'T', 'T', '', '', '', '', 'T', 'T', 'T', 'T']]

In [60]:
def months_to_plant (url): 
    
    html = requests.get(url)
    soup = BeautifulSoup(html.content, "html.parser")

    table = soup.find('table')
    rows = table.find_all('tr')

    name = re.search(r'/([^/?]+)\?', url).group(1)
    zone = re.search(r'zone=(\d+)', url).group(1)

    data = []
    headers = [header.text.strip() for header in rows[0].find_all('th')]

    for row in rows[1:]:
        cells = [cell.text.strip() for cell in row.find_all('td')]
        data.append(cells)

    df_plant = pd.DataFrame(data, columns = headers)
    df_plant ['name'] = name 
    df_plant ['zone'] = zone

    return df_plant 

In [61]:
def concat_multiple_tables(list_):
    
    dfs = []

    for i in list_:
        df = months_to_plant(i)
        if df is not None: 
            dfs.append(df)
    combined_df = pd.concat(dfs, ignore_index = True)

    return combined_df


In [62]:
df_seasonality = concat_multiple_tables(list_url)

AttributeError: 'NoneType' object has no attribute 'find_all'